In [1]:
import numpy as np
import pandas as pd
import math
from itertools import count
from tqdm import tqdm
import threading
import os, re
import multiprocessing
import subprocess
from queue import Empty as QueueEmpty
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
output_dir = 'random_mats'

In [10]:
def generate_mat():
    n = np.random.randint(3,6)
    m = np.random.randint(5,10)
    mat = np.zeros((n,m), dtype=int)
    for i in range(n):
        for j in range(m):
            if np.random.rand() > 0.3:
                mat[i,j] =  np.random.randint(-3,3)
            else:
                mat[i,j] = 0
    return mat

def write_mat(mat, f):
    print(f'{mat.shape[0]} {mat.shape[1]}', file=f)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            print(f'{mat[i,j]} ', end='', file=f)
        print('', file=f)

def read_rows(f):
    l = f.readline()
    return int(l.split(' ')[0])

def generate_and_test(m, regenerate_matrices=True):
    np.random.seed(1234 + m)
    timedout = False

    mat_fname = f'{output_dir}/in{m:03}.mat'
    hil_fname = f'{output_dir}/in{m:03}.hil'
    hilb_fname = f'{output_dir}/in{m:03}.hilb'
    
    # solve the problem in the limited time slot
    while True:
        # generate a problem that admits solutions
        while True:   
            if os.path.exists(hil_fname):
                with open(hil_fname, 'r') as f:
                    nr = read_rows(f)
                # print(f'  {output_dir}/in{m:03} has {nr} solutions.')
                if nr >= 10 and not timedout:
                    break
            
            if regenerate_matrices:
                mat = generate_mat(); timedout = False
                with open(mat_fname, 'w') as f:
                    write_mat(mat, f)

            cmd = f'timeout 10s hilbert {output_dir}/in{m:03} > /dev/null'
            os.system(cmd)

        if os.path.exists(hilb_fname):
            with open(hilb_fname, 'r') as f:
                nr2 = read_rows(f)
                if nr==nr2:
                    return 0 # already tested, same results

        cmd = f'timeout 10s ./build/sym_hilbert -o -c -v -p {output_dir}/in{m:03} -ige | grep "Total S-Vectors processed" '
        r = os.system(cmd) >> 8
        print(f'{output_dir}/in{m:03} exited with code: {r}   # of solutions: {nr}')
        timedout = (r == 124)
        if r != 124 or not regenerate_matrices:
            break # solved in time

    return r

In [11]:

for ii in range(50):
    if generate_and_test(ii, False) not in [0, 124]:
        break

Total S-Vectors processed: 3380
random_mats/in000 exited with code: 0   # of solutions: 279
Total S-Vectors processed: 3349
random_mats/in001 exited with code: 0   # of solutions: 21
Total S-Vectors processed: 138
random_mats/in002 exited with code: 0   # of solutions: 21
Total S-Vectors processed: 126
random_mats/in003 exited with code: 0   # of solutions: 14
Total S-Vectors processed: 9
random_mats/in004 exited with code: 0   # of solutions: 10
Total S-Vectors processed: 110
random_mats/in005 exited with code: 0   # of solutions: 26
Total S-Vectors processed: 98
random_mats/in006 exited with code: 0   # of solutions: 23
Total S-Vectors processed: 147
random_mats/in007 exited with code: 0   # of solutions: 15
Total S-Vectors processed: 1111
random_mats/in008 exited with code: 0   # of solutions: 72
Total S-Vectors processed: 25
random_mats/in009 exited with code: 0   # of solutions: 12
Total S-Vectors processed: 69
random_mats/in010 exited with code: 0   # of solutions: 10
Total S-Vec